In [2]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 25.8 MB/s 
     |████████████████████████████████| 163 kB 72.3 MB/s 
     |████████████████████████████████| 7.6 MB 64.5 MB/s 


In [3]:
import pandas as pd 
from transformers import pipeline

In [4]:
##Zero Shot Classification with Hugging Face

In [5]:
classifier = pipeline("zero-shot-classification", device=0)

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


Downloading:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [9]:
##Get the Text Data¶

In [10]:

import numpy as np


In [13]:
data=pd.read_csv('/content/Dataset - NLP Assignment.csv',encoding='latin-1')

In [15]:
data.head()

,ID,MEMBER_ID,COMMENT
0,a2p1U000000RowfQAC,0011U00000rjFKdQAM,"I showed up for my appointment, but they had m..."
1,a2p1U000000RqQqQAK,0011U00000riCSHQA2,"Staff was polite, courteous, and on time"
2,a2p1U000000RqXyQAK,0011U00000riTw7QAE,Overall care is great! It's wonderful to be a...
3,a2p1U000000Rq1LQAS,0011U00000rhu8eQAA,Like the doctor and staff at this location. Ea...
4,a2p1U000000RpiuQAC,0011U00000rk4SHQAY,The convenience and the doctors


In [16]:
data.describe()

,ID,MEMBER_ID,COMMENT
count,3812,3812,3812
unique,3812,3005,3718
top,a2p1U000000RowfQAC,0011U00000ri0eLQAQ,Convenience
freq,1,10,13


In [17]:
data.shape

(3812, 3)

In [18]:
df_head = data.head(3812)

In [19]:
df_head.head()

,ID,MEMBER_ID,COMMENT
0,a2p1U000000RowfQAC,0011U00000rjFKdQAM,"I showed up for my appointment, but they had m..."
1,a2p1U000000RqQqQAK,0011U00000riCSHQA2,"Staff was polite, courteous, and on time"
2,a2p1U000000RqXyQAK,0011U00000riTw7QAE,Overall care is great! It's wonderful to be a...
3,a2p1U000000Rq1LQAS,0011U00000rhu8eQAA,Like the doctor and staff at this location. Ea...
4,a2p1U000000RpiuQAC,0011U00000rk4SHQAY,The convenience and the doctors


In [20]:
#df_head.head()

In [26]:
samples = data[1:3813]

In [27]:
strings = samples.COMMENT.values

In [28]:
sequence = list(strings)

In [29]:
print(sequence)

['Staff was polite, courteous, and on time', "Overall care is great!  It's wonderful to be able to spend 45-60 minutes with the actual Dr.", 'Like the doctor and staff at this location. Easy to get any appointment.', 'The convenience and the doctors', 'They are very good about returning calls', "I like the convenience of the location to my work and I like one provider who will keep track of all of my other providers etc.  I was sick and they were able to help me and had the PX right there so I didn't have to go to another place to fill my prescription.", 'Appt ease Dr spends time Thorough Close to work', 'I WAS IN A LOT OF DISCOMFORT YESTERDAY AND APPRECIATED THE QUICK RESPONSE', 'Focus on preventative services.  My lipids panel was not included my fasting blood work.', 'Not Sure', 'It is a good set up if you can get it for free through the State, as I can. I was in need of a primary care doctor and wanted an actual doctor, not a nurse practitioner, and I was struggling to find one thr

In [30]:
sentiment_labels = ['positive','negative']

In [31]:
sent = classifier(sequence, sentiment_labels)

In [32]:
sent

[{'sequence': 'Staff was polite, courteous, and on time',
  'labels': ['positive', 'negative'],
  'scores': [0.993431568145752, 0.0065684630535542965]},
 {'sequence': "Overall care is great!  It's wonderful to be able to spend 45-60 minutes with the actual Dr.",
  'labels': ['positive', 'negative'],
  'scores': [0.9980834722518921, 0.0019165596459060907]},
 {'sequence': 'Like the doctor and staff at this location. Easy to get any appointment.',
  'labels': ['positive', 'negative'],
  'scores': [0.9937335848808289, 0.006266347132623196]},
 {'sequence': 'The convenience and the doctors',
  'labels': ['positive', 'negative'],
  'scores': [0.983291506767273, 0.016708485782146454]},
 {'sequence': 'They are very good about returning calls',
  'labels': ['positive', 'negative'],
  'scores': [0.9918531179428101, 0.008146857842803001]},
 {'sequence': "I like the convenience of the location to my work and I like one provider who will keep track of all of my other providers etc.  I was sick and t

In [33]:
##Converting the result into DataFrame with the maximum score

In [34]:
output = []

filter_key1 = ['labels']
filter_key2 = ['scores']

for index, row in samples.iterrows():
  d = {}
  seq = row['COMMENT']
  result = classifier(seq, sentiment_labels, multi_label=True)
 # print(result)
  temp_label = list(map(result.get, filter_key1))
  temp_score = list(map(result.get, filter_key2))
 # print(temp)
  d['COMMENT'] = row['COMMENT']
  d['label'] = temp_label[0][0]
  d['score'] = temp_score[0][0]
  output.append(d)

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:1046: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


In [35]:
 #convert the list of dictionary into pandas DataFrame
data_new = pd.DataFrame(output)
data_new


,COMMENT,label,score
0,"Staff was polite, courteous, and on time",positive,0.997987
1,Overall care is great! It's wonderful to be a...,positive,0.998474
2,Like the doctor and staff at this location. Ea...,positive,0.996352
3,The convenience and the doctors,positive,0.990139
4,They are very good about returning calls,positive,0.992499
...,...,...,...
3806,yes,positive,0.995313
3807,Very skeptical that you will soon be without a...,negative,0.871457
3808,"Doctor Malfese is easily accessible, staff is ...",positive,0.997193
3809,Friendly Staff that can be reached at any time,positive,0.989496


In [36]:
data_new.shape

(3811, 3)

In [37]:
data_new.to_csv('data_new.csv')